In [1]:
import pandas as pd
import numpy as np


In [23]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [25]:
df = pd.read_csv('covid_toy.csv')

In [27]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [29]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [31]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [33]:
from sklearn.model_selection import train_test_split

In [35]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']), df['has_covid'], test_size=0.2)

In [37]:
X_train

,age,gender,fever,cough,city
22,71,Female,98.0,Strong,Kolkata
59,6,Female,104.0,Mild,Kolkata
44,20,Male,102.0,Strong,Delhi
98,5,Female,98.0,Strong,Mumbai
65,69,Female,102.0,Mild,Bangalore
...,...,...,...,...,...
49,44,Male,104.0,Mild,Mumbai
31,83,Male,103.0,Mild,Kolkata
32,34,Female,101.0,Strong,Delhi
42,27,Male,100.0,Mild,Delhi


# How Programers used to do this before Column Transformers

In [43]:
# adding simple imputer to fever column
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

#also the test data
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [51]:
# Ordinal Encoding -> cough
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

#also the test datq
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [57]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first')
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [61]:
# Extacting Age
X_train_age = X_train.drop(columns=['gender','fever', 'cough', 'city']).values

# also on the test data
X_test_age = X_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

X_train_age.shape

(80, 1)

In [67]:

X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 2 has 0 dimension(s)

# How Do they Do now...

In [72]:
from sklearn.compose import ColumnTransformer

In [76]:
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(drop='first'), ['gender', 'city'])
], remainder='passthrough')

In [78]:
transformer.fit_transform(X_train).shape

(80, 7)

In [84]:
transformer.transform(X_test).shape

(20, 7)